In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
# focus!! if you find you can not use the global_variables_initializer, it is because you do not use the same version 
# as the author use, you can upgrade the tensorflow or change the old one to 'initialize_all_variables', it can also 
# work well!
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Loss at step 0: 21.712219
Training accuracy: 6.7%
Validation accuracy: 7.5%
Loss at step 100: 2.304909
Training accuracy: 71.3%
Validation accuracy: 70.3%
Loss at step 200: 1.839097
Training accuracy: 74.5%
Validation accuracy: 73.0%
Loss at step 300: 1.585855
Training accuracy: 76.1%
Validation accuracy: 74.0%
Loss at step 400: 1.417213
Training accuracy: 77.3%
Validation accuracy: 74.8%
Loss at step 500: 1.295023
Training accuracy: 78.1%
Validation accuracy: 75.2%
Loss at step 600: 1.200985
Training accuracy: 78.8%
Validation accuracy: 75.5%
Loss at step 700: 1.125530
Training accuracy: 79.4%
Validation accuracy: 75.7%
Loss at step 800: 1.063089
Training accuracy: 79.8%
Validation accuracy: 75.8%
Test accuracy: 83.0%


In [14]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


In [15]:
# change the same thing! 'global_variables_initgializer'

num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))



Initialized
Minibatch loss at step 0: 19.199926
Minibatch accuracy: 10.0%
Validation accuracy: 16.9%
Minibatch loss at step 500: 1.838635
Minibatch accuracy: 70.0%
Validation accuracy: 70.2%
Minibatch loss at step 1000: 0.261689
Minibatch accuracy: 90.0%
Validation accuracy: 75.9%
Minibatch loss at step 1500: 1.044846
Minibatch accuracy: 70.0%
Validation accuracy: 74.7%
Minibatch loss at step 2000: 0.018482
Minibatch accuracy: 100.0%
Validation accuracy: 75.3%
Minibatch loss at step 2500: 1.201182
Minibatch accuracy: 70.0%
Validation accuracy: 74.5%
Minibatch loss at step 3000: 1.796094
Minibatch accuracy: 70.0%
Validation accuracy: 75.2%
Minibatch loss at step 3500: 2.421975
Minibatch accuracy: 80.0%
Validation accuracy: 75.5%
Minibatch loss at step 4000: 0.491872
Minibatch accuracy: 90.0%
Validation accuracy: 76.7%
Minibatch loss at step 4500: 1.170256
Minibatch accuracy: 70.0%
Validation accuracy: 73.9%
Minibatch loss at step 5000: 2.123490
Minibatch accuracy: 80.0%
Validation accur

In [17]:
# add a hidden layer with 1024 nodes, the input data is the same as the network without hidden  layer.
batch_size = 128
hidden_layer = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. here!! we have two weights array here, the first is the input to the hidden , the 
  # the second the the hidden layer to the output layer, and also the biases.
  weights_input_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer]))
  biases_hidden = tf.Variable(tf.zeros([hidden_layer]))
  weights_hidden_output = tf.Variable(
    tf.truncated_normal([hidden_layer, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation. !!notice the hidden layer, using the nn.relu() function to do the math
  # the hidden layer output
  logits_hidden = tf.matmul(tf_train_dataset, weights_input_hidden) + biases_hidden
  # then the output layer output
  logits_output = tf.matmul(tf.nn.relu(logits_hidden), weights_hidden_output) + biases_output
  # finally, we computate the loss of the whole net
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_output, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_output)
  hidden_logits_valid = tf.matmul(tf_valid_dataset, weights_input_hidden) + biases_hidden
  output_logits_valid = tf.matmul(tf.nn.relu(hidden_logits_valid), weights_hidden_output) + biases_output
  valid_prediction = tf.nn.softmax(output_logits_valid)
  hidden_logits_test = tf.matmul(tf_test_dataset, weights_input_hidden) + biases_hidden
  output_logits_test = tf.matmul(tf.nn.relu(hidden_logits_test), weights_hidden_output) + biases_output
  test_prediction = tf.nn.softmax(output_logits_test)


In [19]:
num_steps = 7001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 313.451782
Minibatch accuracy: 9.4%
Validation accuracy: 24.9%
Minibatch loss at step 500: 24.832773
Minibatch accuracy: 75.0%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 11.313919
Minibatch accuracy: 84.4%
Validation accuracy: 82.9%
Minibatch loss at step 1500: 17.665604
Minibatch accuracy: 82.8%
Validation accuracy: 83.1%
Minibatch loss at step 2000: 12.711016
Minibatch accuracy: 79.7%
Validation accuracy: 82.1%
Minibatch loss at step 2500: 5.374044
Minibatch accuracy: 85.9%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 4.436986
Minibatch accuracy: 85.9%
Validation accuracy: 84.5%
Minibatch loss at step 3500: 3.875599
Minibatch accuracy: 85.9%
Validation accuracy: 84.7%
Minibatch loss at step 4000: 7.823914
Minibatch accuracy: 82.8%
Validation accuracy: 82.5%
Minibatch loss at step 4500: 4.029365
Minibatch accuracy: 89.1%
Validation accuracy: 84.6%
Minibatch loss at step 5000: 2.279006
Minibatch accuracy: 85.9%
Validation ac

In [ ]:
# FIRST SITUATION :even i just use the LogisticRrgression and one hidden layer we can get 90% accuracy, 
# here are the parameters i choose batch_size=128, step number = 5001, learning rate= 0.5, hidden node= 1024
# SECOND SITUATION : i continuely enlarge the step number = 7001, 
# i got the accuracy 90.1%, almost the same as the step number = 5001
# THIRD SITUATION : still remain the step number , i desrease the learning rate , from  0.5 to 0.2
# i got the accuracy 90.4% , seems like got a higher accuracy 
# FOURTH SITUATION : desrease the learning rate from 0.2 to 0.1 , 
# believe it or not , i got a higher accuracy--->90.7%